In [23]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline

from config import mapbox_api_key, b_api_key

import pandas as pd

import requests

from pprint import pprint
from datetime import datetime as dt
from datetime import timedelta

# Functions to retreive data:

In [3]:
def num_hours_ago(hours):
    """
    Returns UTC time x hours ago.
    """
#     from datetime import datetime as dt
    date = dt.utcnow() - timedelta(hours=hours)
    return dt.strftime(date, "%Y-%m-%dT%H:%M:%S")

In [4]:
def get_coords(location):
    """
    Returns coordinates of address for given input.
    """
    from config import g_api_key
    base_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={location}&key={g_api_key}"
    
    data = requests.get(base_url).json()
    
    try:
        loc = data['results'][0]['geometry']['location']
        
        lat = loc['lat']
        lng = loc['lng']
    except:
        return "404"
    
    return lat,lng

In [5]:
def pollution(lat,lng):
    """
    Returns Breezometer pollution data from coordinates.
    """
    
    end_datetime = num_hours_ago(1)
    start_datetime = num_hours_ago(168)
    
    try:
        base_url = f"https://api.breezometer.com/air-quality/v2/historical/hourly?lat={lat}&lon={lng}&key={b_api_key}&start_datetime={start_datetime}&end_datetime={end_datetime}"
        data = requests.get(base_url).json()
        return data
    except:
        return None

In [68]:
def pollution_epa(lat,lng):
    """
    Returns EPA pollution data from coordinates.
    """
    
    base_url = f"https://api.breezometer.com/air-quality/v2/current-conditions?lat={lat}&lon={lng}&key={b_api_key}&features=local_aqi"
    data = requests.get(base_url).json()
    return data

# Scatterplot function:

In [6]:
def generate_plot(city):  
    '''
    Generates plot HTML for AQI for past week.
    '''
    try:    
        lat,lng = get_coords(city)
    except ValueError:
        return "Error: We could not find the coordinates for the entry you supplied."

    pollution_data = pollution(lat,lng)
    
    if pollution_data != None:
        try:
            x = [x['datetime'] for x in pollution_data['data'] if x['data_available'] == True]
            y = [y['indexes']['baqi']['aqi'] for y in pollution_data['data'] if y['data_available'] == True]

            text1 = [text['indexes']['baqi']['dominant_pollutant'] for text in pollution_data['data'] if text['data_available'] == True]
            text2 = [text['indexes']['baqi']['category'] for text in pollution_data['data'] if text['data_available'] == True]

            test = [f"Dominant pollutant: {x}\n{y}" for x,y in zip(text1,text2)]

            data = [go.Scattergl(x=x,
                               y=y, 
                               mode="markers", 
                               marker=dict(
                                size=12,
                                color = y, #set color equal to a variable
                                colorscale='Viridis',
                                showscale=True
                               ),  
                               name="AQI",
                                text= test)]

            layout = go.Layout(title=f"AQI for {city.title()}",
                            xaxis=dict(title='Date'),
                            yaxis=dict(title='AQI'))

            fig = go.Figure(data=data, layout=layout)
            
            return py.plot(fig, filename='jupyter-basic_line',auto_open=False)
        
        except:
            return "404"        

# Mapping functions:

In [71]:
def df_geodata(filename=None,query=None):    
    """
    Returns dataframe with pollution data from city list dataframe. Makes API calls via 'pollution_epa'
    to scrape latest data.
    """
    
    if filename and query:
        return "You must pass either a csv filename or a SQL query. You cannot pass two arguments."
    elif filename == None and query == None:
        return "Error: No arguments were passed to this function."
    elif filename:
        df = pd.read_csv(filename)
    else:
        df = pd.read_sql_query(query).set_index('index')
    
    aqi = []
    category = []
    dominant_pollutant = []
    date = []


    for lat,lng in zip(df['lat'],df['lng']):
        # Get pollution data.
        data = pollution_epa(lat,lng)
        
        if data != None:
            index = data['data']['indexes']['usa_epa']

            air_quality = index['aqi']
            categories = index['category']
            dom_pollutant = index['dominant_pollutant']
            datetime = data['data']['datetime']

            aqi.append(air_quality)
            category.append(categories)
            dominant_pollutant.append(dom_pollutant)
            date.append(datetime)
            
        else:
            aqi.append("NaN")
            category.append("NaN")
            dominant_pollutant.append("NaN")
            date.append("NaN")          
    
    # Update columns with latest data
    df['aqi'] = aqi
    df['category'] = category
    df['dominant_pollutant'] = dominant_pollutant
    df['datetime'] = date
    
    return df

In [167]:
def marker_text(df):
    '''
    Generates hover-over text for AQI info.
    '''
    text = []
    for row in df.itertuples():
        a = getattr(row, "city")
        b = getattr(row, "category")
        c = getattr(row, "aqi")
        d = getattr(row, "dominant_pollutant")
        string = f"{a}<br>{b}<br>AQI: {c}<br>Dominant pollutant: {d}"
        text.append(string)
    return text

In [200]:
def generate_map(df):
    '''
    Generates HTML for map from pollution data.
    '''
    data = [
    go.Scattermapbox(
        lat=round(df['lat'],3),
        lon=round(df['lng'],3),
        mode='markers',
        marker=dict(
            size=df['rain']*10,
            color= df['rain'],
            colorscale = 'Blues',
            reversescale=True
        ),
#         text= marker_text(df)
        )
    ]

    layout = go.Layout(
        autosize=True,
        hovermode='closest',
        mapbox=dict(
            accesstoken=mapbox_api_key,
            bearing=0,
            center=dict(
                lat=36,
                lon=-119
            ),
            style='dark',
            pitch=0,
            zoom=4
        ),
        margin=dict(
        l= 0,
        r= 0,
        b= 0,
        t= 0,
        pad= 2
        )
    )

    fig = dict(data=data, layout=layout)

#     map_html = plotly.offline.plot(fig, include_plotlyjs=False, output_type='div')
    plot = py.iplot(fig, filename='test_map')
    
    return plot

In [177]:
df = pd.read_csv('california_cities.csv')
df.head()

,city,type,county,state,lat,lng,uv_value,aqi,category,dominant_pollutant,Date,Temperature,Cloud,Pressure,Wind(speed),rain
0,Adelanto,City,San Bernardino,california,34.582770,-117.409215,2.98,41,Good air quality,o3,2018-12-01 06:06:27+00,8.91,75,1015.0,5.1,0.00
1,Agoura Hills,City,Los Angeles,california,34.153340,-118.761676,3.00,48,Good air quality,pm25,2018-12-01 06:06:27+00,14.48,40,1016.0,3.6,0.00
2,Alameda,City,Alameda,california,37.765206,-122.241636,2.00,61,Moderate air quality,pm25,2018-12-01 06:06:27+00,10.26,1,1020.0,2.1,1.02
3,Albany,City,Alameda,california,37.886870,-122.297748,1.97,61,Moderate air quality,pm25,2018-12-01 06:06:27+00,9.75,1,1020.0,2.1,1.02
4,Alhambra,City,Los Angeles,california,34.095287,-118.127015,3.00,66,Moderate air quality,pm25,2018-12-01 06:06:27+00,13.80,1,1015.0,2.1,0.00


In [201]:
generate_map(df)

# Appendix:

In [7]:
# x,y = get_coords("beijing")

In [8]:
# print(x)
# pollution_data = pollution(x,y)

In [9]:
# generate_plot("san francisco")

# x = [x['datetime'] for x in pollution_data['data'] if x['data_available'] == True]
# y = [y['indexes']['baqi']['aqi'] for y in pollution_data['data'] if y['data_available'] == True]
# text1 = [text['indexes']['baqi']['dominant_pollutant'] for text in pollution_data['data'] if text['data_available'] == True]
# text2 = [text['indexes']['baqi']['category'] for text in pollution_data['data'] if text['data_available'] == True]

# test = [f"Dominant pollutant: {x}\n{y}" for x,y in zip(text1,text2)]

# data = [go.Scattergl(x=x,
#                    y=y, 
#                    mode="markers", 
#                    marker=dict(
#                     size=10,
#                     color = y, #set color equal to a variable
#                     colorscale='Viridis',
#                     showscale=True,
#                     reversescale=True
#                    ),  
#                    name="AQI",
#                     text= test)]

# layout = go.Layout(title="AQI for beijing",
#                 xaxis=dict(title='Date'),
#                 yaxis=dict(title='AQI'))

# fig = go.Figure(data=data, layout=layout)

# # py.iplot(fig, filename='jupyter-basic_line')

# p = plotly.offline.plot(fig, include_plotlyjs=False, output_type='div')

In [10]:
# print(p)
# print(text1)
# print(text2)
# print(test)
# x,y = "404"
# print(p)

In [15]:

# df_wind = pd.read_csv('https://plot.ly/~datasets/2805.csv')

# df_known_capacity = df_wind[ df_wind['total_cpcy'] != -99999.000 ]
# df_sum = df_known_capacity.groupby('manufac')['total_cpcy'].sum().sort_values(ascending=False).to_frame()

# df_farms = pd.read_csv('https://plot.ly/~jackp/17256.csv')
# df_farms.set_index('Wind Farm', inplace=True)

# wind_farms=list([
#     dict(
#         args=[ { 
#             'mapbox.center.lat':38,
#             'mapbox.center.lon':-94,
#             'mapbox.zoom':3,
#             'annotations[0].text':'All US wind turbines (scroll to zoom)'
#         } ],
#         label='USA',
#         method='relayout'
#     )
# ])

# for farm, row in df_farms.iterrows():
#     desc = []
#     for col in df_farms.columns:
#         if col not in ['DegMinSec','Latitude','Longitude']:
#             if str(row[col]) not in ['None','nan','']: 
#                 desc.append( col + ': ' + str(row[col]).strip("'") )
#     desc.insert(0, farm)
#     wind_farms.append( 
#         dict(
#             args=[ { 
#                 'mapbox.center.lat':row['Latitude'], 
#                 'mapbox.center.lon':float(str(row['Longitude']).strip("'")), 
#                 'mapbox.zoom':9,
#                 'annotations[0].text': '<br>'.join(desc)
#             } ],
#             label=' '.join(farm.split(' ')[0:2]),
#             method='relayout'
#         )
#     )

# data = []
# for mfr in list(df_sum.index):
#     if mfr != 'unknown':
#         trace = dict(
#             lat = df_wind[ df_wind['manufac'] == mfr ]['lat_DD'],
#             lon = df_wind[ df_wind['manufac'] == mfr ]['long_DD'],
#             name = mfr,
#             marker = dict(size = 4),
#             type = 'scattermapbox'
#         )
#     data.append(trace)

# # mapbox_access_token = 'insert mapbox token here'

# layout = dict(
#     height = 800,
#     margin = dict( t=0, b=0, l=0, r=0 ),
#     font = dict( color='#FFFFFF', size=11 ),
#     paper_bgcolor = '#000000',
#     mapbox=dict(
#         accesstoken=mapbox_api_key,
#         bearing=0,
#         center=dict(
#             lat=38,
#             lon=-94
#         ),
#         pitch=0,
#         zoom=3,
#         style='dark'
#     ),
# )

# updatemenus=list([
#     dict(
#         buttons = wind_farms[0:10],
#         pad = {'r': 0, 't': 10},
#         x = 0.1,
#         xanchor = 'left',
#         y = 1.0,
#         yanchor = 'top',
#         bgcolor = '#AAAAAA',
#         active = 99,
#         bordercolor = '#FFFFFF',
#         font = dict(size=11, color='#000000')
#     ),
#     dict(
#         buttons=list([
#             dict(
#                 args=['mapbox.style', 'dark'],
#                 label='Dark',
#                 method='relayout'
#             ),                    
#             dict(
#                 args=['mapbox.style', 'light'],
#                 label='Light',
#                 method='relayout'
#             ),
#             dict(
#                 args=['mapbox.style', 'satellite'],
#                 label='Satellite',
#                 method='relayout'
#             ),
#             dict(
#                 args=['mapbox.style', 'satellite-streets'],
#                 label='Satellite with Streets',
#                 method='relayout'
#             )                    
#         ]),
#         direction = 'up',
#         x = 0.75,
#         xanchor = 'left',
#         y = 0.05,
#         yanchor = 'bottom',
#         bgcolor = '#000000',
#         bordercolor = '#FFFFFF',
#         font = dict(size=11)
#     ),        
# ])

# annotations = list([
#     dict(text='All US wind turbines (scroll to zoom)', font=dict(color='magenta',size=14), borderpad=10, 
#          x=0.05, y=0.05, xref='paper', yref='paper', align='left', showarrow=False, bgcolor='black'),
#     dict(text='Wind<br>Farms', x=0.01, y=0.99, yref='paper', align='left', showarrow=False,font=dict(size=14))
# ])

# layout['updatemenus'] = updatemenus
# layout['annotations'] = annotations

# figure = dict(data=data, layout=layout)
# py.iplot(figure, filename='wind-turbine-territory-dropdown')

In [110]:
# data = [
#     go.Scattermapbox(
#         lat=round(df['lat'],3),
#         lon=round(df['lng'],3),
#         mode='markers',
#         marker=dict(
#             size=df['aqi']/10,
#             color= df['aqi'],
#             colorscale = 'Jet',
#         ),
#         text= marker_text(df)
#     )
# ]

# layout = go.Layout(
#     autosize=True,
#     hovermode='closest',
#     mapbox=dict(
#         accesstoken=mapbox_api_key,
#         bearing=0,
#         center=dict(
#             lat=36,
#             lon=-119
#         ),
#         style='dark',
#         pitch=4,
#         zoom=4
#     ),

#     margin=dict(
#     l= 0,
#     r= 0,
#     b= 0,
#     t= 0,
#     pad= 2
#     )
# )

# fig = dict(data=data, layout=layout)

# py.iplot(fig, filename='CA Mapbox')
# py.iplot(fig, filename='CA Mapbox')
# cali_plot = plotly.offline.plot(fig, include_plotlyjs=False, output_type='div')

In [103]:
# print(cali_plot)